# Example Data Science Notebook

This notebook demonstrates basic data analysis workflows.

In [ ]:
import sys
import os
from pathlib import Path


# Add src to path
# sys.path.insert(0, str(Path().resolve().parent / 'src'))

sys.path.append(str(ROOT / "src"))
os.environ["SPARK_LOCAL_DIRS"] = str((ROOT / "spark_tmp").resolve())

import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns

from ds.analyzer import DataAnalyzer
from de.load.loader import DataLoader

print("Imports successful!")

## Load Sample Data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Create sample data
np.random.seed(42)
data = {
    'feature1': np.random.randn(100),
    'feature2': np.random.randn(100),
    'feature3': np.random.randn(100),
    'target': np.random.randint(0, 2, 100)
}
df = pd.DataFrame(data)
df.head()

## Exploratory Data Analysis

In [ ]:
analyzer = DataAnalyzer(df)
analyzer.summary_stats()

In [ ]:
# Check for missing values
analyzer.check_missing()

## Visualizations

In [ ]:
# Distribution plot
plt.figure(figsize=(12, 4))
for i, col in enumerate(['feature1', 'feature2', 'feature3'], 1):
    plt.subplot(1, 3, i)
    plt.hist(df[col], bins=20, edgecolor='black')
    plt.title(f'Distribution of {col}')
    plt.xlabel('Value')
    plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

In [ ]:
# Correlation matrix
plt.figure(figsize=(8, 6))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix')
plt.show()

## Prepare Data for ML

In [ ]:
X_train, X_test, y_train, y_test = analyzer.prepare_features('target')
print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col,
    expr,
    lower,
    regexp_replace,
    to_timestamp,
)
from pathlib import Path

from src.common.config import RAW_DIR
from src.de.spark_jobs.traffy_basic_etl import main

RAW_FILE = Path("../data/raw/bangkok_traffy.csv")
OUTPUT_DIR = Path("../data/preprocessed/traffy_cleaned_parquet")

In [ ]:
import os, sys
from pathlib import Path

ROOT = Path.cwd().parent
sys.path.append(str(ROOT / "src"))
os.environ["SPARK_LOCAL_DIRS"] = str((ROOT / "spark_tmp").resolve())

from src.de.spark_jobs import traffy_basic_etl as etl

etl.main()


In [ ]:
from pathlib import Path
out = Path("data/processed/traffy_cleaned_parquet")
if out.exists():
    if out.is_file():
        out.unlink()
    else:
        import shutil
        shutil.rmtree(out)


In [1]:
%pip install findspark scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-21-openjdk-amd64"
os.environ["SPARK_HOME"] = "/opt/spark"

In [ ]:
# import os
# os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-21"
# os.environ["SPARK_HOME"] = "C:\spark"
# os.environ["HADOOP_HOME"] = "C:\hadoop"

In [5]:
import findspark
findspark.init()

In [6]:
spark_url = 'local'

In [7]:
from pyspark.sql import SparkSession
import os

spark = SparkSession.builder\
        .master(spark_url)\
        .appName('Spark Tutorial')\
        .config('spark.ui.port', '4040')\
        .getOrCreate()

spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/20 00:28:28 WARN Utils: Your hostname, sira, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/11/20 00:28:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/20 00:28:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
cd ..

/home/sirav/JekTurnRight_dsde


/home/sirav/spark_env/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [13]:
from src.de.spark_jobs.traffy_flood_etl import run_traffy_flood_etl
import os
from pathlib import Path

ROOT = Path.cwd()

INPUT = f"{ROOT}/data/raw/bangkok_traffy.csv"
CLEANED_OUT = f"{ROOT}/data/processed/traffy_clean.parquet"
FLOOD_TS_OUT = f"{ROOT}/data/processed/flood_daily_by_district.parquet"

run_traffy_flood_etl(
    spark,
    input_path=INPUT,
    cleaned_output_path=CLEANED_OUT,
    flood_ts_output_path=FLOOD_TS_OUT
)
spark.stop()


🚀 RUNNING TRAFFY FLOOD ETL PIPELINE
[STEP] Loading raw Traffy CSV...


[STEP] Cleaning + schema validation + Bangkok filter...
[STEP] Adding time columns...
[STEP] Adding flood_flag column...
[STEP] Aggregating daily flood counts per district...
[STEP] Writing cleaned tickets...


25/11/20 00:31:00 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[STEP] Writing daily flood time series...


✅ ETL COMPLETED
   Cleaned data: /home/sirav/JekTurnRight_dsde/data/processed/traffy_clean.parquet
   Flood TS: /home/sirav/JekTurnRight_dsde/data/processed/flood_daily_by_district.parquet


In [11]:
%pip install pandas matplotlib seaborn scikit-learn requests

IOStream.flush timed out
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 2.1 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 KB 2.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 KB 1.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 2.0 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.9/113.9 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.0/325.0 KB 5.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 4.5 MB/s eta 0:00:00a 0:00:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 2.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.6/153.6 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 KB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [14]:
# use spark read this /home/sirav/JekTurnRight_dsde/data/processed/traffy_clean.parquet
os.environ["SPARK_HOME"] = "/opt/spark"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-21-openjdk-amd64"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("JekTurnRight Analysis") \
    .getOrCreate()
df = spark.read.parquet("/home/sirav/JekTurnRight_dsde/data/processed/traffy_clean.parquet")
df.show(5)

df.printSchema()
df.count()
df.select("district").distinct().show()
df.groupBy("district").count().orderBy("count", ascending=False).show()
df.filter(df["district"] == "บางรัก").show(5)
df.createOrReplaceTempView("traffy_data")
# result = spark.sql("SELECT district, COUNT(*) as report_count FROM traffy_data GROUP BY
#     district ORDER BY report_count DESC")
# result.show()



+-----------+-------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+-----------+--------+-------------+---------+----+------------+--------------------+--------------------+----------+----+-----+---+----+-------+--------+
|  ticket_id|         type|        organization|             comment|               photo|         photo_after|            coords|             address|subdistrict|district|     province|    state|star|count_reopen|       last_activity|           timestamp|      date|year|month|day|hour|weekday|is_flood|
+-----------+-------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+-----------+--------+-------------+---------+----+------------+--------------------+--------------------+----------+----+-----+---+----+-------+--------+
|2021-4D9Y98|           {}|เขตลาดพร้าว,การไฟ...|หน้าปากซอย ลาดพร้...|https://storage.

In [19]:
# read /home/sirav/JekTurnRight_dsde/data/processed/flood_daily_by_district.parquet
flood_df = spark.read.parquet("/home/sirav/JekTurnRight_dsde/data/processed/flood_daily_by_district.parquet")
# flood_df.show()

# sort by date
flood_df = flood_df.orderBy("date")
flood_df.show()

+----------+-----------+---------------------+
|      date|   district|flood_complaint_count|
+----------+-----------+---------------------+
|2021-09-19|     ประเวศ|                    1|
|2021-12-22|     ประเวศ|                    1|
|2022-01-14|       สาทร|                    1|
|2022-02-04|       สาทร|                    1|
|2022-02-26|      ดุสิต|                    1|
|2022-03-03|   ตลิ่งชัน|                    1|
|2022-04-01|      บางนา|                    1|
|2022-05-18|   ลาดพร้าว|                    1|
|2022-05-20|   ลาดพร้าว|                    1|
|2022-05-23|     บางเขน|                    1|
|2022-05-24|   ลาดพร้าว|                    1|
|2022-05-25|   ลาดพร้าว|                    3|
|2022-05-26|    จตุจักร|                    1|
|2022-05-26|    บึงกุ่ม|                    1|
|2022-05-27|   ห้วยขวาง|                    1|
|2022-05-28|บางขุนเทียน|                    1|
|2022-05-28| วังทองหลาง|                    1|
|2022-05-28|    ราชเทวี|                    1|
|2022-05-28| 

In [21]:
from pathlib import Path
from src.common.config import PROJECT_ROOT, RAW_DIR
print("FILE:", Path(__file__).resolve())
print("PROJECT_ROOT:", PROJECT_ROOT)
print("RAW_DIR:", RAW_DIR)


NameError: name '__file__' is not defined